<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
diff --git a/build.sh b/build.sh
index 6addf18..cafc92a 100755
--- a/build.sh
+++ b/build.sh
@@ -492,11 +492,8 @@ write_commit_file() {
 build_lite_x86_64_jni_and_jar()
 {
     # copy x86 so
-    local is_train=off
+    local is_train=on
     cd ${BASEPATH}/output/tmp
-    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
-      is_train=on
-    fi
     local pkg_name=mindspore-lite-${VERSION_STR}-linux-x64
 
     cd ${BASEPATH}/output/tmp/
@@ -506,7 +503,12 @@ build_lite_x86_64_jni_and_jar()
     rm -rf ${LITE_JAVA_PATH}/native/libs/linux_x86/ && mkdir -pv ${LITE_JAVA_PATH}/native/libs/linux_x86/
     cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/java/linux_x86/libs/
     cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/native/libs/linux_x86/
-    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+    local train_so=$pkg_name/runtime/lib/libmindspore-lite-train.so
+    if [ ! -f "$train_so" ]; then
+      echo "not exist"
+      is_train=off
+    fi
+    if [ "$is_train" == on ]; then
         cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/linux_x86/libs/
         cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/linux_x86/
     fi
@@ -641,10 +643,7 @@ build_lite_arm64_and_jni() {
     # build arm64
     build_lite "arm64"
     # copy arm64 so
-    local is_train=off
-    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
-      is_train=on
-    fi
+    local is_train=on
     local pkg_name=mindspore-lite-${VERSION_STR}-android-aarch64
     cd "${BASEPATH}/mindspore/lite/build"
 
@@ -654,7 +653,12 @@ build_lite_arm64_and_jni() {
     rm -rf ${LITE_JAVA_PATH}/native/libs/arm64-v8a/   && mkdir -p ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
     cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
     cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
-    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+    local train_so=$pkg_name/runtime/lib/libmindspore-lite-train.so
+    if [ ! -f "$train_so" ]; then
+      echo "not exist"
+      is_train=off
+    fi
+    if [ "$is_train" == on ]; then
       cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
       cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
     fi
@@ -679,10 +683,7 @@ build_lite_arm32_and_jni() {
     # build arm32
     build_lite "arm32"
     # copy arm32 so
-    local is_train=off
-    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
-      is_train=on
-    fi
+    local is_train=on
     local pkg_name=mindspore-lite-${VERSION_STR}-android-aarch32
     cd "${BASEPATH}/mindspore/lite/build"
 
@@ -692,7 +693,12 @@ build_lite_arm32_and_jni() {
     rm -rf ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/   && mkdir -pv ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
     cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
     cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
-    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+    local train_so=$pkg_name/runtime/lib/libmindspore-lite-train.so
+    if [ ! -f "$train_so" ]; then
+      echo "not exist"
+      is_train=off
+    fi
+    if [ "$is_train" == on ]; then
       cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
       cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
     fi
diff --git a/mindspore/lite/test/st/scripts/run_benchmark_arm.sh b/mindspore/lite/test/st/scripts/run_benchmark_arm.sh
index c1cc3eb..cc2e5f3 100644
--- a/mindspore/lite/test/st/scripts/run_benchmark_arm.sh
+++ b/mindspore/lite/test/st/scripts/run_benchmark_arm.sh
@@ -102,14 +102,14 @@ function Run_Converter() {
           continue
         fi
         echo ${model_name}'_train' >> "${run_converter_log_file}"
-        echo './converter_lite  --fmk=MINDIR --modelFile='${models_path}'/'${model_name}' --outputFile='${ms_models_path}'/'${model_name}'_train  --trainModel=true' >> "${run_converter_log_file}"
-        ./converter_lite  --fmk=MINDIR --modelFile=${models_path}/${model_name} --outputFile=${ms_models_path}/${model_name}'_train' --trainModel=true
+        echo './converter_lite  --fmk=MINDIR --modelFile='${train_models_path}'/'${model_name}' --outputFile='${ms_models_path}'/'${model_name}'_train  --trainModel=true' >> "${run_converter_log_file}"
+        ./converter_lite  --fmk=MINDIR --modelFile=${train_models_path}/${model_name} --outputFile=${train_modes_path}/${model_name}'_train' --trainModel=true
         if [ $? = 0 ]; then
             converter_result='converter mindspore '${model_name}'_train pass';echo ${converter_result} >> ${run_converter_result_file}
         else
             converter_result='converter mindspore '${model_name}'_train failed';echo ${converter_result} >> ${run_converter_result_file};return 1
         fi
-    done < ${models_mindspore_train_config}
+    done < ${models_ms_train_config}
 
     # Convert TFLite PostTraining models:
     while read line; do
@@ -606,13 +606,17 @@ function Run_arm64() {
     cd ${benchmark_test_path} || exit 1
     if [ -f ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ]; then
         cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+        cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/libjpeg-turbo/lib/libjpeg.so* ${benchmark_test_path}/ || exit 1
+        cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/libjpeg-turbo/lib/libturbojpeg.so* ${benchmark_test_path}/ || exit 1
     fi
     cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
     cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
     cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
 
     cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version_arm}-android-aarch64/runtime/lib/libmindspore-lite-train.so ${benchmark_train_test_path}/libmindspore-lite-train.so || exit 1
     cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/tools/benchmark_train/benchmark_train ${benchmark_test_path}/benchmark_train || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -621,6 +625,7 @@ function Run_arm64() {
     echo 'cd  /data/local/tmp/benchmark_test' > adb_cmd.txt
     echo 'cp  /data/local/tmp/libc++_shared.so ./' >> adb_cmd.txt
     echo 'chmod 777 benchmark' >> adb_cmd.txt
+    echo 'chmod 777 benchmark_train' >> adb_cmd.txt
 
     adb -s ${device_id} shell < adb_cmd.txt
 
@@ -1016,6 +1021,63 @@ function Run_arm64() {
             run_result='arm64: '${model_name}' failed'; echo ${run_result} >> ${run_benchmark_result_file}; return 1
         fi
     done < ${models_for_process_only_config}
+    fail=0
+    # Run mindir converted train models:
+    tmp_dir=/data/local/tmp/benchmark_train_test
+    adb -s ${device_id} push ${benchmark_train_test_path} $tmp_dir > adb_push_log.txt
+    while read line; do
+        LFS=" " read -r -a line_array <<< ${line}
+        model_prefix=${line_array[0]}
+        model_name=${line_array[0]}'_train'
+        accuracy_limit=0.5
+        if [[ $model_name == \#* ]]; then
+            continue
+        fi
+        if [[ "${line_array[1]}" == "weight_quant" ]]; then
+            model_name=${line_array[0]}'_train_quant'
+            accuracy_limit=${line_array[2]}
+        fi
+        export_file="${tmp_dir}/${model_name}_tod"
+        inference_file="${tmp_dir}/${model_name}_infer"
+
+        if [[ "${line_array[1]}" == "noarm32" ]] && [[ "$1" == arm32 ]]; then
+            run_result=$1': '${model_name}' irrelevant'; echo ${run_result} >> ${run_benchmark_train_result_file}
+            continue
+        fi
+        # run benchmark_train test with clib data
+        echo ${model_name} >> "${run_arm64_fp32_log_file}"
+        adb -s ${device_id} push ${train_io_path}/${model_prefix}_input*.bin ${train_io_path}/${model_prefix}_output*.bin /data/local/tmp/benchmark_train_test >> adb_push_log.txt
+        echo 'cd /data/local/tmp/benchmark_train_test' > ${adb_cmd_arm64_run_file}
+        echo 'chmod 777 benchmark_train' >> ${adb_cmd_arm64_run_file}
+        echo 'cp  /data/local/tmp/libc++_shared.so ./' >> ${adb_cmd_arm64_run_file}
+        adb -s ${device_id} shell < ${adb_cmd_arm64_run_file} >> ${run_arm64_fp32_log_file}
+        echo "rm -f ${export_file}* ${inference_file}*" >> ${run_arm64_fp32_log_file}
+        echo "rm -f ${export_file}* ${inference_file}*" >> ${adb_cmd_arm64_run_file}
+        adb -s ${device_id} shell < ${adb_cmd_arm64_run_file} >> ${run_arm64_fp32_log_file}
+        adb_cmd=$(cat <<-ENDM
+        export LD_LIBRARY_PATH=./:/data/local/tmp/:/data/local/tmp/benchmark_train_test;./benchmark_train \
+        --epochs=${epoch_num} \
+        --modelFile=${model_name}.ms \
+        --inDataFile=${tmp_dir}/${model_prefix}_input \
+        --expectedDataFile=${tmp_dir}/${model_prefix}_output \
+        --numThreads=${threads} \
+        --accuracyThreshold=${accuracy_limit} \
+        --inferenceFile=${inference_file} \
+        --exportFile=${export_file}
+ENDM
+        )
+        echo "${adb_cmd}" >> ${run_arm64_fp32_log_file}
+        echo "${adb_cmd}" >> ${adb_cmd_arm64_run_file}
+        adb -s ${device_id} shell < ${adb_cmd_arm64_run_file} >> ${run_arm64_fp32_log_file}
+        # TODO: change to arm_type
+        if [ $? = 0 ]; then
+            run_result=$1': '${model_name}' pass'; echo ${run_result} >> ${run_benchmark_train_result_file}
+        else
+            run_result=$1': '${model_name}' failed'; echo ${run_result} >> ${run_benchmark_train_result_file};
+            fail=1
+        fi
+    done < ${models_ms_train_config}
+    return ${fail}
 }
 
 # Run on arm32 platform:
@@ -1027,12 +1089,16 @@ function Run_arm32() {
     cd ${benchmark_test_path} || exit 1
     if [ -f ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libminddata-lite.so ]; then
         cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+        cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/libjpeg-turbo/lib/libturbojpeg.so* ${benchmark_test_path}/ || exit 1
+        cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/libjpeg-turbo/lib/libjpeg.so* ${benchmark_test_path}/ || exit 1
     fi
     cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
     cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
     cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
-    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite-train.so || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libmindspore-lite-train.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
     cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/tools/benchmark_train/benchmark_train ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -1072,6 +1138,65 @@ function Run_arm32() {
             run_result='arm32: '${model_name}' failed'; echo ${run_result} >> ${run_benchmark_result_file}; return 1
         fi
     done < ${models_arm32_config}
+
+    fail=0
+    # Run mindir converted train models:
+    tmp_dir=/data/local/tmp/benchmark_train_test
+    # cp train ms to adb
+    adb -s ${device_id} push ${benchmark_train_test_path} $tmp_dir > adb_push_log.txt
+    while read line; do
+        LFS=" " read -r -a line_array <<< ${line}
+        model_prefix=${line_array[0]}
+        model_name=${line_array[0]}'_train'
+        accuracy_limit=0.5
+        if [[ $model_name == \#* ]]; then
+            continue
+        fi
+        if [[ "${line_array[1]}" == "weight_quant" ]]; then
+            model_name=${line_array[0]}'_train_quant'
+            accuracy_limit=${line_array[2]}
+        fi
+        export_file="${tmp_dir}/${model_name}_tod"
+        inference_file="${tmp_dir}/${model_name}_infer"
+
+        if [[ "${line_array[1]}" == "noarm32" ]] && [[ "$1" == arm32 ]]; then
+            run_result=$1': '${model_name}' irrelevant'; echo ${run_result} >> ${run_benchmark_train_result_file}
+            continue
+        fi
+        # run benchmark_train test without clib data
+        echo ${model_name} >> "${run_arm32_log_file}"
+        adb -s ${device_id} push ${train_io_path}/${model_prefix}_input*.bin ${train_io_path}/${model_prefix}_output*.bin  /data/local/tmp/benchmark_train_test >> adb_push_log.txt
+        echo 'cd /data/local/tmp/benchmark_train_test' > ${adb_cmd_arm32_run_file}
+        echo 'chmod 777 benchmark_train' >> ${adb_cmd_arm32_run_file}
+        echo 'cp  /data/local/tmp/libc++_shared.so ./' >> ${adb_cmd_arm32_run_file}
+        adb -s ${device_id} shell < adb_run_cmd.txt >> ${run_arm32_log_file}
+        echo "rm -f ${export_file}* ${inference_file}*" >> ${run_arm32_log_file}
+        echo "rm -f ${export_file}* ${inference_file}*" >> ${adb_cmd_arm32_run_file}
+        adb -s ${device_id} shell < ${adb_cmd_arm32_run_file} >> ${run_arm32_log_file}
+        adb_cmd=$(cat <<-ENDM
+        export LD_LIBRARY_PATH=./:/data/local/tmp/:/data/local/tmp/benchmark_train_test;./benchmark_train \
+        --epochs=${epoch_num} \
+        --modelFile=${model_name}.ms \
+        --inDataFile=${tmp_dir}/${model_prefix}_input \
+        --expectedDataFile=${tmp_dir}/${model_prefix}_output \
+        --numThreads=${threads} \
+        --accuracyThreshold=${accuracy_limit} \
+        --inferenceFile=${inference_file} \
+        --exportFile=${export_file}
+ENDM
+        )
+        echo "${adb_cmd}" >> ${run_arm32_log_file}
+        echo "${adb_cmd}" >> ${adb_cmd_arm32_run_file}
+        adb -s ${device_id} shell < ${adb_cmd_arm32_run_file} >> ${run_arm32_log_file}
+        # TODO: change to arm_type
+        if [ $? = 0 ]; then
+            run_result=$1': '${model_name}' pass'; echo ${run_result} >> ${run_benchmark_train_result_file}
+        else
+            run_result=$1': '${model_name}' failed'; echo ${run_result} >> ${run_benchmark_train_result_file};
+            fail=1
+        fi
+    done < ${models_ms_train_config}
+    return ${fail}
 }
 
 # Run on arm64-fp16 platform:
@@ -1456,7 +1581,8 @@ function Print_Benchmark_Result() {
 basepath=$(pwd)
 echo ${basepath}
 #set -e
-
+train_models_path=""
+train_io_path=""
 # Example:sh run_benchmark_arm.sh -r /home/temp_test -m /home/temp_test/models -d "8KE5T19620002408" -e arm_cpu
 while getopts "r:m:d:e:" opt; do
     case ${opt} in
@@ -1468,6 +1594,14 @@ while getopts "r:m:d:e:" opt; do
             models_path=${OPTARG}
             echo "models_path is ${OPTARG}"
             ;;
+        i)
+            train_io_path=${OPTARG}
+            echo "train_io_path is ${OPTARG}"
+            ;;
+        M)
+            train_models_path=${OPTARG}
+            echo "train_models_path is ${models_path}"
+            ;;
         d)
             device_id=${OPTARG}
             echo "device_id is ${OPTARG}"
@@ -1476,6 +1610,10 @@ while getopts "r:m:d:e:" opt; do
             backend=${OPTARG}
             echo "backend is ${OPTARG}"
             ;;
+        q)
+           threads=${OPTARG}
+           echo "threads=${threads}"
+           ;;
         ?)
         echo "unknown para"
         exit 1;;
@@ -1514,9 +1652,32 @@ models_with_multiple_inputs_config=${basepath}/../config/models_with_multiple_in
 models_for_process_only_config=${basepath}/../config/models_for_process_only.cfg
 models_tf_weightquant_config=${basepath}/../config/models_tf_weightquant.cfg
 models_codegen_config=${basepath}/../codegen/models_codegen.cfg
+models_ms_train_config=${basepath}/../config/models_ms_train.cfg
 
 ms_models_path=${basepath}/ms_models
+train_modes_path = ${basepath}/ms_train_models
+rm -rf ${train_modes_path}
+mkdir -p ${train_modes_path}
 build_path=${basepath}/codegen_build
+logs_path=${basepath}/logs_train
+rm -rf ${logs_path}
+mkdir -p ${logs_path}
+adb_cmd_arm64_run_file = ${logs_path}/adb_arm64_cmd.txt
+adb_cmd_arm32_run_file = ${logs_path}/adb_arm32_cmd.txt
+
+train_models_path
+if [[ train_models_path == "" ]]
+then
+  echo "train_io path is empty"
+  train_models_path="../${models_path}/models_train/"
+fi
+echo train_models_path
+if [[ $train_io_path == "" ]]
+then
+  echo "train_io path is empty"
+  train_io_path=${train_models_path}/input_output
+fi
+echo $train_io_path
 
 # Write converter result to temp file
 run_converter_log_file=${basepath}/run_converter_log.txt
@@ -1555,6 +1716,9 @@ echo 'run arm64_fp32 logs: ' > ${run_arm64_fp32_log_file}
 run_arm64_fp32_codegen_log_file=${basepath}/run_arm64_fp32_codegen_log.txt
 echo 'run arm64_codegen logs: ' > ${run_arm64_fp32_codegen_log_file}
 
+run_arm64_train_fp32_log_file=${basepath}/run_arm64_train_fp32_log.txt
+echo 'run arm64_train_fp32 logs: ' > ${run_arm64_train_fp32_log_file}
+
 run_arm32_fp32_codegen_log_file=${basepath}/run_arm32_fp32_codegen_log.txt
 echo 'run arm32_codegen logs: ' > ${run_arm32_fp32_codegen_log_file}
 
@@ -1572,9 +1736,13 @@ echo "Push files to the arm and run benchmark"
 benchmark_test_path=${basepath}/benchmark_test
 rm -rf ${benchmark_test_path}
 mkdir -p ${benchmark_test_path}
+benchmark_train_test_path = =${basepath}/benchmark_train_test
+rm -rf ${benchmark_train_test_path}
+mkdir -p ${benchmark_train_test_path}
 cp -a ${ms_models_path}/*.ms ${benchmark_test_path} || exit 1
 # Copy models converted using old release of mslite converter for compatibility test
 cp -a ${models_path}/compatibility_test/*.ms ${benchmark_test_path} || exit 1
+cp -a {train_modes_path}/*.ms ${benchmark_train_test_path} || exit 1
 
 backend=${backend:-"all"}
 isFailed=0
@@ -1646,6 +1814,20 @@ if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_fp32" ]
     sleep 1
 fi
 
+if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_fp32" || $backend == "train"]]; then
+    # Run on arm64
+    arm64_path=${release_path}/android_aarch64
+    # mv ${arm64_path}/*train-android-aarch64* ./train
+    file_name=$(ls ${arm64_path}/*android-aarch64.tar.gz)
+    IFS="-" read -r -a file_name_array <<< "$file_name"
+    version=${file_name_array[2]}
+
+    echo "start Run arm64 train ..."
+    Run_arm64
+    Run_arm64_fp32_status=$?
+    sleep 1
+fi
+
 if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_fp16" ]]; then
     # Run on arm64-fp16
     arm64_path=${release_path}/android_aarch64
@@ -1675,6 +1857,13 @@ if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_codegen
         isFailed=1
     fi
 fi
+if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_train" ]]; then
+    if [[ ${Run_arm64_train} != 0 ]];then
+        echo "Run_arm64_train failed"
+        cat ${run_arm64_fp32_train_log_file}
+        isFailed=1
+    fi
+fi
 if [[ $backend == "all" || $backend == "arm32_cpu" || $backend == "arm32_codegen" ]]; then
     if [[ ${Run_arm32_codegen_status} != 0 ]];then
         echo "Run_arm32 codegen failed"
diff --git a/mindspore/lite/test/st/scripts/run_benchmark_x86.sh b/mindspore/lite/test/st/scripts/run_benchmark_x86.sh
index ebeed83..5f8aab1 100644
--- a/mindspore/lite/test/st/scripts/run_benchmark_x86.sh
+++ b/mindspore/lite/test/st/scripts/run_benchmark_x86.sh
@@ -1,5 +1,36 @@
 #!/bin/bash
 
+# Run Export on x86 platform and create output test files:
+docker_image=mindspore_build:210301
+function Run_Export(){
+    cd $models_path || exit 1
+    if [[ -z "${CLOUD_MODEL_ZOO}" ]]; then
+        echo "CLOUD_MODEL_ZOO is not defined - exiting export models"
+        exit 1
+    fi
+    # Export mindspore train models:
+    while read line; do
+        LFS=" " read -r -a line_array <<< ${line}
+        model_name=${line_array[0]}
+        if [[ $model_name == \#* ]]; then
+          continue
+        fi
+        echo ${model_name}'_train_export.py' >> "${export_log_file}"
+        echo 'exporting' ${model_name}
+        if [ -n "$docker_image" ]; then
+          echo 'docker run --user '"$(id -u):$(id -g)"' --env CLOUD_MODEL_ZOO=${CLOUD_MODEL_ZOO} -w $PWD --runtime=nvidia -v /home/$USER:/home/$USER -v /opt/share:/opt/share --privileged=true '${docker_image}' python '${models_path}'/'${model_name}'_train_export.py' >>  "${export_log_file}"
+          docker run --user "$(id -u):$(id -g)" --env CLOUD_MODEL_ZOO=${CLOUD_MODEL_ZOO} -w $PWD --runtime=nvidia -v /home/$USER:/home/$USER -v /opt/share:/opt/share --privileged=true "${docker_image}" python ${models_path}'/'${model_name}_train_export.py "${epoch_num}"
+        else
+          echo 'CLOUD_MODEL_ZOO=${CLOUD_MODEL_ZOO} python '${models_path}'/'${model_name}'_train_export.py' >>  "${export_log_file}"
+          CLOUD_MODEL_ZOO=${CLOUD_MODEL_ZOO} python ${models_path}'/'${model_name}_train_export.py "${epoch_num}"
+        fi
+        if [ $? = 0 ]; then
+            export_result='export mindspore '${model_name}'_train_export pass';echo ${export_result} >> ${export_result_file}
+        else
+            export_result='export mindspore '${model_name}'_train_export failed';echo ${export_result} >> ${export_result_file}
+        fi
+    done < ${models_ms_train_config}
+}
 # Run converter on x86 platform:
 function Run_Converter() {
     # Unzip x86 runtime and converter
@@ -97,17 +128,25 @@ function Run_Converter() {
 
     # Convert mindspore train models:
     while read line; do
-        model_name=${line}
+        LFS=" " read -r -a line_array <<< ${line}
+        WEIGHT_QUANT=""
+        model_prefix=${line_array[0]}'_train'
+        model_name=${line_array[0]}'_train'
         if [[ $model_name == \#* ]]; then
           continue
         fi
-        echo ${model_name}'_train' >> "${run_converter_log_file}"
-        echo './converter_lite  --fmk=MINDIR --modelFile='${models_path}'/'${model_name}' --outputFile='${ms_models_path}'/'${model_name}'_train  --trainModel=true' >> "${run_converter_log_file}"
-        ./converter_lite  --fmk=MINDIR --modelFile=${models_path}/${model_name} --outputFile=${ms_models_path}/${model_name}'_train' --trainModel=true
+        if [[ "${line_array[1]}" == "weight_quant" ]]; then
+            WEIGHT_QUANT="--quantType=WeightQuant --bitNum=8 --quantWeightSize=0 --quantWeightChannel=0"
+            model_name=${line_array[0]}'_train_quant'
+        fi
+
+        echo ${model_name} >> "${run_converter_log_file}"
+        echo './converter_lite  --fmk=MINDIR --modelFile='${train_models_path}'/'${model_prefix}'.mindir --outputFile='${ms_models_path}'/'${model_name}' --trainModel=true' ${WEIGHT_QUANT} >> "${run_converter_log_file}"
+        ./converter_lite --fmk=MINDIR --modelFile=${train_models_path}/${model_prefix}.mindir --outputFile=${ms_models_path}/${model_name} --trainModel=true ${WEIGHT_QUANT}
         if [ $? = 0 ]; then
-            converter_result='converter mindspore '${model_name}'_train pass';echo ${converter_result} >> ${run_converter_result_file}
+            converter_result='converter mindspore '${model_name}' pass';echo ${converter_result} >> ${run_converter_result_file}
         else
-            converter_result='converter mindspore '${model_name}'_train failed';echo ${converter_result} >> ${run_converter_result_file};return 1
+            converter_result='converter mindspore '${model_name}' failed';echo ${converter_result} >> ${run_converter_result_file};return 1
         fi
     done < ${models_mindspore_train_config}
 
@@ -346,7 +385,7 @@ function Run_Converter() {
 function Run_x86() {
     echo 'cd  '${x86_path}'/mindspore-lite-'${version}'-linux-x64' >> "${run_x86_log_file}"
     cd ${x86_path}/mindspore-lite-${version}-linux-x64 || return 1
-    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./runtime/lib
+    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./runtime/lib:./runtime/third_party/libjpeg-turbo/lib
     cp tools/benchmark/benchmark ./ || exit 1
 
     # Run tf converted models:
@@ -666,6 +705,40 @@ function Run_x86() {
             run_result='x86: '${model_name}' failed'; echo ${run_result} >> ${run_benchmark_result_file}; return 1
         fi
     done < ${models_for_process_only_config}
+
+    # Run mindspore converted train models:
+    fail=0
+    while read line; do
+        LFS=" " read -r -a line_array <<< ${line}
+        model_prefix=${line_array[0]}
+        model_name=${line_array[0]}'_train'
+        accuracy_limit=0.5
+        if [[ $model_name == \#* ]]; then
+          continue
+        fi
+        if [[ "${line_array[1]}" == "weight_quant" ]]; then
+            model_name=${line_array[0]}'_train_quant'
+            accuracy_limit=${line_array[2]}
+        fi
+        export_file="${ms_models_path}/${model_name}_tod"
+        inference_file="${ms_models_path}/${model_name}_infer"
+        rm -f ${inference_file}"*"
+        rm -f ${export_file}"*"
+        echo ${model_name} >> "${run_x86_log_file}"
+        ${run_valgrind}./tools/benchmark_train/benchmark_train \
+        --modelFile=${ms_models_path}/${model_name}.ms \
+        --inDataFile=${train_io_path}/${model_prefix}_input \
+        --expectedDataFile=${train_io_path}/${model_prefix}_output --epochs=${epoch_num} --numThreads=${threads} \
+        --accuracyThreshold=${accuracy_limit} --inferenceFile=${inference_file} \
+        --exportFile=${export_file} >> "${run_x86_log_file}"
+        if [ $? = 0 ]; then
+            run_result='x86: '${model_name}' pass'; echo ${run_result} >> ${run_benchmark_train_result_file}
+        else
+            run_result='x86: '${model_name}' failed'; echo ${run_result} >> ${run_benchmark_train_result_file}
+            fail=1
+        fi
+    done < ${models_ms_train_config}
+    return ${fail}
 }
 
 # Run on x86 sse platform:
@@ -1339,10 +1412,42 @@ function Print_Benchmark_Result() {
     MS_PRINT_TESTCASE_END_MSG
 }
 
+function Print_Benchmark_Train_Result() {
+    MS_PRINT_TESTCASE_START_MSG
+    while read line; do
+        arr=("${line}")
+        printf "%-20s %-100s %-7s\n" ${arr[0]} ${arr[1]} ${arr[2]}
+    done < ${run_benchmark_train_result_file}
+    MS_PRINT_TESTCASE_END_MSG
+}
+
 basepath=$(pwd)
 echo ${basepath}
 #set -e
+logs_path=${basepath}/logs_train
+rm -rf ${logs_path}
+mkdir -p ${logs_path}
+# Export model if enabled
+if [[ $enable_export == 1 ]]; then
+    echo "Start Exporting models ..."
+    # Write export result to temp file
+    export_log_file=${logs_path}/export_log.txt
+    echo ' ' > ${export_log_file}
+
+    export_result_file=${logs_path}/export_result.txt
+    echo ' ' > ${export_result_file}
+    # Run export
+    Run_Export
+    Print_Result ${export_result_file}
 
+fi
+
+# Write benchmark_train result to temp file
+run_benchmark_train_result_file=${logs_path}/run_benchmark_train_result.txt
+echo ' ' > ${run_benchmark_train_result_file}
+
+train_models_path=""
+train_io_path=""
 # Example:sh run_benchmark_x86.sh -r /home/temp_test -m /home/temp_test/models -e arm_cpu
 while getopts "r:m:e:" opt; do
     case ${opt} in
@@ -1354,10 +1459,27 @@ while getopts "r:m:e:" opt; do
             models_path=${OPTARG}
             echo "models_path is ${OPTARG}"
             ;;
+        M)
+            train_models_path=${OPTARG}
+            echo "train_models_path is ${models_path}"
+            ;;
         e)
             backend=${OPTARG}
             echo "backend is ${OPTARG}"
             ;;
+        i)
+            train_io_path=${OPTARG}
+            echo "train_io_path is ${OPTARG}"
+            ;;
+        v)
+            run_valgrind="valgrind --log-file=valgrind.log "
+            echo "Run x86 with valgrind"
+            ;;
+        p)
+            enable_export=1
+            docker_image=${OPTARG}
+            echo "enable_export = 1, docker_image = ${OPTARG}"
+            ;;
         ?)
         echo "unknown para"
         exit 1;;
@@ -1390,10 +1512,24 @@ models_for_process_only_config=${basepath}/../config/models_for_process_only.cfg
 models_tf_weightquant_config=${basepath}/../config/models_tf_weightquant.cfg
 models_codegen_config=${basepath}/../codegen/models_codegen.cfg
 models_codegen_parallel_config=${basepath}/../codegen/models_codegen_parallel.cfg
+models_ms_train_config=${basepath}/../config/models_ms_train.cfg
 
 ms_models_path=${basepath}/ms_models
 build_path=${basepath}/codegen_build
 build_parallal_path=${basepath}/codegen_parallel_build
+train_models_path
+if [[ train_models_path == "" ]]
+then
+  echo "train_io path is empty"
+  train_models_path="../${models_path}/models_train/"
+fi
+echo train_models_path
+if [[ $train_io_path == "" ]]
+then
+  echo "train_io path is empty"
+  train_io_path=${train_models_path}/input_output
+fi
+echo $train_io_path
 
 # Write converter result to temp file
 run_converter_log_file=${basepath}/run_converter_log.txt
@@ -1564,4 +1700,9 @@ Print_Benchmark_Result
 if [[ $isFailed == 1 ]]; then
     exit 1
 fi
+echo "Run x86 train end"
+Print_Benchmark_Train_Result
+if [[ $isFailed == 1 ]]; then
+    exit 1
+fi
 exit 0
diff --git a/scripts/lite_release_package.sh b/scripts/lite_release_package.sh
index 7e4e77f..442f59b 100644
--- a/scripts/lite_release_package.sh
+++ b/scripts/lite_release_package.sh
@@ -50,78 +50,32 @@ function android_release_package()
 {
     arch=$1
     device=$2
-    src_inference_pkg_name="mindspore-lite-${version}-inference-android-${arch}"
-    src_train_pkg_name="mindspore-lite-${version}-train-android-${arch}"
     dst_pkg_name="mindspore-lite-${version}-android-${arch}"
-
-    rm -rf ${src_inference_pkg_name}
-    rm -rf ${src_train_pkg_name}
     rm -rf ${dst_pkg_name}
-    tar -xzf ${input_path}/android_${arch}/${device}/${src_inference_pkg_name}.tar.gz
-    tar -xzf ${input_path}/android_${arch}/${device}/${src_train_pkg_name}.tar.gz
-
-    cp -r ${src_train_pkg_name}/tools/benchmark_train/ ${src_inference_pkg_name}/tools/
-    cp -r ${src_train_pkg_name}/train/ ${src_inference_pkg_name}/
-    mkdir -p ${output_path}/release/android/${device}/
-    mv ${src_inference_pkg_name} ${dst_pkg_name}
-    # Copy java runtime to Android package
-    cp ${input_path}/aar/avx/mindspore-lite-*maven*.zip ${dst_pkg_name}
-    tar -czf ${output_path}/release/android/${device}/${dst_pkg_name}.tar.gz ${dst_pkg_name}
+    mv ${input_path}/android_${arch}/${device}/${dst_pkg_name}.tar.gz ${output_path}/release/android/${device}/${dst_pkg_name}.tar.gz
     cd ${output_path}/release/android/${device}/
     sha256sum ${dst_pkg_name}.tar.gz > ${dst_pkg_name}.tar.gz.sha256
-    cd -
-
-    verify_every_file ${src_train_pkg_name}/tools/benchmark_train/ ${dst_pkg_name}
-    verify_every_file ${src_train_pkg_name}/train/ ${dst_pkg_name}
-
-    rm -rf ${src_train_pkg_name}
-    rm -rf ${dst_pkg_name}
 }
 
 function linux_release_package()
 {
-    src_inference_pkg_name="mindspore-lite-${version}-inference-linux-x64"
-    src_train_pkg_name="mindspore-lite-${version}-train-linux-x64"
-    dst_pkg_name="mindspore-lite-${version}-linux-x64"
 
-    rm -rf ${src_inference_pkg_name}
-    rm -rf ${src_train_pkg_name}
+    dst_pkg_name="mindspore-lite-${version}-linux-x64"
     rm -rf ${dst_pkg_name}
-    tar -xzf ${input_path}/ubuntu_x86/avx/${src_inference_pkg_name}.tar.gz
-    tar -xzf ${input_path}/ubuntu_x86/${src_train_pkg_name}.tar.gz
-
-    cp -r ${src_train_pkg_name}/tools/benchmark_train/ ${src_inference_pkg_name}/tools/
-    cp -r ${src_train_pkg_name}/train/ ${src_inference_pkg_name}/
-
     mkdir -p ${output_path}/release/linux/
-    mv ${src_inference_pkg_name} ${dst_pkg_name}
-    tar -czf ${output_path}/release/linux/${dst_pkg_name}.tar.gz ${dst_pkg_name}
+    mv ${input_path}/ubuntu_x86/${dst_pkg_name}.tar.gz ${output_path}/release/linux/
     cd ${output_path}/release/linux/
     sha256sum ${dst_pkg_name}.tar.gz > ${dst_pkg_name}.tar.gz.sha256
-    cd -
-
-    verify_every_file ${src_train_pkg_name}/tools/benchmark_train/ ${dst_pkg_name}
-    verify_every_file ${src_train_pkg_name}/train/ ${dst_pkg_name}
-    rm -rf ${src_train_pkg_name}
-    rm -rf ${dst_pkg_name}
 }
 
 function windows_release_package()
 {
-    src_inference_pkg_name="mindspore-lite-${version}-inference-win-x64"
-    dst_pkg_name="mindspore-lite-${version}-win-x64"
+    pkg_name="mindspore-lite-${version}-win-x64"
 
-    rm -rf ${src_inference_pkg_name}
-    rm -rf ${dst_pkg_name}
-    unzip ${input_path}/windows_x64/avx/${src_inference_pkg_name}.zip
-
-    mv ${src_inference_pkg_name} ${dst_pkg_name}
-    mkdir -p ${output_path}/release/windows/
-    zip -r ${output_path}/release/windows/${dst_pkg_name}.zip ${dst_pkg_name}
+    rm -rf ${pkg_name}
+    mv  ${input_path}/windows_x64/avx/${pkg_name}.zip ${output_path}/release/windows/${dst_pkg_name}.zip
     cd ${output_path}/release/windows/
     sha256sum ${dst_pkg_name}.zip > ${dst_pkg_name}.zip.sha256
-    cd -
-    rm -rf ${dst_pkg_name}
 }
 
 echo "============================== begin =============================="
